In [1]:
import requests
import pandas
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.supabase import SupabaseVectorStore
from supabase.client import Client, create_client
import datetime
from tqdm import tqdm

import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
papersDF = pandas.read_csv('csv-microRNAAN-set (1).csv')

In [3]:
embedding = OpenAIEmbeddings()
supabase: Client = create_client(supabase_url=os.environ.get("SUPABASE_URL"), supabase_key=os.environ.get("SUPABASE_SERVICE_KEY"))

In [4]:
loader_not_works = []
no_url = []

In [6]:
pbar = tqdm(total=len(papersDF))
for index, row in papersDF.iterrows():
    pbar.update(1)
    if index != 0:
        pbar.set_description(f"{(len(no_url) + len(loader_not_works)) / index * 100}% failure")
    if index <= 4911:
        continue
    doi = row['DOI']
    # print(f"https://api.unpaywall.org/v2/{doi}?email=ryandu9221@gmail.com")
    req = requests.get(f"https://api.unpaywall.org/v2/{doi}?email=ryandu9221@gmail.com")
    if req.status_code == 200:
        if req.json().get('is_oa') == True:
            oa = req.json().get('best_oa_location')
            if oa == None:
                no_url.append(doi)
                # print("no oa " + doi)
                continue
            url = oa.get('url_for_pdf')
            if url is None:
                no_url.append(doi)
                # print("no url " + doi)
                continue
            try:
                fileloader = OnlinePDFLoader(url)
                loader = UnstructuredPDFLoader(fileloader.file_path)
                splitter = RecursiveCharacterTextSplitter(
                    chunk_size = 500,
                    chunk_overlap  = 0,
                    length_function = len,
                    is_separator_regex = False,
                )
                chunks = loader.load_and_split(text_splitter=splitter)
                for chunk in chunks:
                    chunk.metadata["doi"] = doi
                    chunk.metadata["add_date"] = str(datetime.datetime.now().date())
                    chunk.metadata.pop("source")
                embedded = SupabaseVectorStore.from_documents(chunks, embedding, client=supabase, table_name="knowledge")
            except:
                loader_not_works.append(doi)
                # print("not work " + doi)
    else:
        no_url.append(doi)
    
pbar.close()

23.237863091610805% failure: 100%|██████████| 4924/4924 [05:01<00:00, 16.34it/s]


In [6]:
req.json()


{'doi': '10.3390/genes13050747',
 'doi_url': 'https://doi.org/10.3390/genes13050747',
 'title': 'Genetic Variants of MIR27A, MIR196A2 May Impact the Risk for the Onset of Coronary Artery Disease in the Pakistani Population',
 'genre': 'journal-article',
 'is_paratext': False,
 'published_date': '2022-04-24',
 'year': 2022,
 'journal_name': 'Genes',
 'journal_issns': '2073-4425',
 'journal_issn_l': '2073-4425',
 'journal_is_oa': True,
 'journal_is_in_doaj': True,
 'publisher': 'MDPI AG',
 'is_oa': True,
 'oa_status': 'gold',
 'has_repository_copy': True,
 'best_oa_location': {'updated': '2023-08-13T18:05:58.604486',
  'url': 'https://www.mdpi.com/2073-4425/13/5/747/pdf?version=1650788366',
  'url_for_pdf': 'https://www.mdpi.com/2073-4425/13/5/747/pdf?version=1650788366',
  'url_for_landing_page': 'https://doi.org/10.3390/genes13050747',
  'evidence': 'oa journal (via doaj)',
  'license': 'cc-by',
  'version': 'publishedVersion',
  'host_type': 'publisher',
  'is_best': True,
  'pmh_id':

# Test Cycle

In [ ]:
test_doi = "10.1038/s41584-020-0426-0"

In [ ]:
req = requests.get(f"https://api.unpaywall.org/v2/{test_doi}?email=ryandu9221@gmail.com")
url = req.json().get('best_oa_location').get('url')
fileloader = OnlinePDFLoader(url)
loader = UnstructuredPDFLoader(fileloader.file_path)
splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 500,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = False,
)
chunks = loader.load_and_split(text_splitter=splitter)
for chunk in chunks:
    chunk.metadata["doi"] = test_doi
    chunk.metadata["add_date"] = str(datetime.datetime.now().date())
    chunk.metadata.pop("source")

In [ ]:
embedded = SupabaseVectorStore.from_documents(chunks, embedding, client=supabase, table_name="knowledge")

AttributeError: 'tuple' object has no attribute 'page_content'